In [2]:
# Libraries
# library containign functions that read and write to csv files
import lib.handle_csv as csvh
# library for connecting to the db
import lib.handle_db as dbh
# library for handling text matchings
import lib.text_comp as txtc
# library for getting data from crossref
import lib.crossref_api as cr_api
# library for handling url searchs
import lib.handle_urls as urlh

from pathlib import Path


# input files
new_results_file = 'repo_searches/diamond_pub_search.csv'
previous_results = 'pop_searches/ukch_pop_prev_res.csv'

#output files
nr_wf = new_results_file[:-4]+"_wf.csv"
working_file = wf_fields = None
current_pass = 0
if Path(nr_wf).is_file():
    working_file, wf_fields = csvh.get_csv_data(nr_wf,'Num')
    for art_num in working_file:
        if current_pass < int(working_file[art_num]['ignore']) :
            current_pass = int(working_file[art_num]['ignore'])

In [3]:
if current_pass == 0:
    i = 0
    db_conn = dbh.DataBaseAdapter('ukch_articles.sqlite')
    for art_num in working_file:
        if working_file[art_num]['ignore']=='0':
            new_title = working_file[art_num]['Title']
            new_doi = working_file[art_num]['DOI'].lower()
            db_title = db_conn.get_title(new_doi)
            if db_title == None:
                print("Not in DB:", new_doi, new_title)
            else:
                print("Already in DB:", new_doi, "for:", new_title, db_title)
                working_file[art_num]['ignore'] = '5'
    print("without DOI:", i)
    csvh.write_csv_data(working_file, nr_wf)
    current_pass = 5

In [18]:
if current_pass >= 5:
    i = 0
    for art_num in working_file:
        if working_file[art_num]['ignore']=='0':
            article_title = working_file[art_num]['Title']
            article_doi = working_file[art_num]['DOI']
            article_url = "" #working_file[art_num]['ArticleURL']
            print("Analysing:", article_title, article_doi, article_url)
            # try to retrive html page for article using link from crossref first
            # and if not try url from pop
            # find reference to uk catalysis hub in html text
            # if found mark as relevant
            found = ""
            referents = ["uk catalysis hub", "uk catalysis", "catalysis hub",
                 'EP/R026645/1', 'resources', 'EP/K014668/1', 'EPSRC', 'EP/K014714/1',
                 'Hub','provided', 'grant', 'biocatalysis', 'EP/R026815/1', 'EP/R026939/1',
                 'support', 'membership', 'EP/M013219/1', 'UK', 'kindly', 'Catalysis',
                 'funded', 'EP/R027129/1', 'Consortium', 'thanked', 'EP/K014854/1', 'EP/K014706/2']
            found = urlh.findFromDOI(article_title, article_doi, referents)
            working_file[art_num]['checked_doi'] = 1
            working_file[art_num]['ack_doi'] = found
            #found = urlh.findFromURI(article_title, article_url, referents)
            #working_file[art_num]['checked_url'] = 1
            #working_file[art_num]['ack_url'] = found
            print("Ack:", found)
    csvh.write_csv_data(working_file, nr_wf)

Analysing: Synchrotron radiation and catalytic science 10.1080/08940886.2020.1701368 
Ack: 
Analysing: Rewiring the ‘push-pull’ catalytic machinery of a heme enzyme using an expanded genetic code 10.1021/acscatal.9b05129 
Ack: <span class="article_header-suppInfo-text">Supporting Info (1)</span>
Analysing: Correlation of the ratio of metallic to oxide species with activity of PdPt catalysts for methane oxidation 10.1039/C9CY02371B 
Ack: 
Analysing: Understanding the mechanochemical synthesis of the perovskite LaMnO 3 and its catalytic behaviour 10.1039/C9DT03590G 
Ack: <span>
                            UK Catalysis Hub, Research Complex at Harwell, Rutherford Appleton Laboratories, Harwell Science &amp; Innovation Campus, Didcot, Oxfordshire OX11 0FA, UK

                                <br>
<b>E-mail:</b>
<a href="mailto:P.P.Wells@soton.ac.uk" title="P.P.Wells@soton.ac.uk">P.P.Wells@soton.ac.uk</a> </br></span>
Analysing: Influence of synthesis conditions on the structure of nickel n